In [1]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'): 
        os.chdir('../')
set_root_path()
sys.path.append('python/')
sys.path = [p for p in sys.path if "unbiased-inverse-volume-rendering" not in p]
print(os.getcwd())
print(sys.executable)
os.environ.pop("PYTHONPATH")

c:\Users\mayuf\source\repos\practical-inverse-rendering-of-textured-and-translucent-appearance
c:\Users\mayuf\anaconda3\envs\practical-inverse-rendering\python.exe


'C:\\Users\\mayuf\\source\\repos\\unbiased-inverse-volume-rendering\\mitsuba3\\build\\Release\\python'

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import gridspec
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import mitsuba as mi
import drjit as dr
import numpy as np

from practical_reconstruction import figutils
from practical_reconstruction import scene_configuration
from practical_reconstruction import io_utils
from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

mi.set_variant("cuda_ad_rgb")

from core import integrators
from core import bsdfs
from core import textures
from core import emitters

integrators.register()
bsdfs.register()
textures.register()
emitters.register()

In [3]:
def load_modified_scene(scene_config, res=[400, 240]):
  os.makedirs(scene_config.result_folder,exist_ok=True)

  tmp_mitsuba_xml = io_utils.mitsuba_remote_to_local(scene_config,override=True)
  scene = mi.load_file(tmp_mitsuba_xml)
  params = mi.traverse(scene)

  params.update()
  return scene

In [ ]:
def _update_light_position(params, camera_idx):
	# Update light position for the selected sensor
	light_positions = [
		["6.500000 0.000000 0.000000 0.300000 0.000000 7.471460 0.043578 2.949553 0.000000 -0.653668 0.498097 -35.366814 0.000000 0.000000 0.000000 1.000000"],
		["6.500000 0.000000 0.000000 0.300000 0.000000 6.797308 -0.211309 20.791634 0.000000 3.169637 0.453154 -32.220776 0.000000 0.000000 0.000000 1.000000"],
		["6.351649 -0.439473 0.102093 -6.846509 -0.000000 7.209023 0.137921 -3.654459 -1.380779 -2.021596 0.469633 -33.374290 0.000000 0.000000 0.000000 1.000000"],
		["6.351649 0.385103 0.103064 -6.914496 -0.000000 7.277604 -0.120858 14.460037 -1.380779 1.771490 0.474100 -33.687031 0.000000 0.000000 0.000000 1.000000"],
		["6.495393 -0.088264 -0.017879 1.551554 -0.000000 7.124097 -0.156308 16.941559 0.244696 2.342958 0.474603 -33.722225 0.000000 0.000000 0.000000 1.000000"],
		["6.495393 -0.210534 -0.012542 1.177927 -0.000000 4.997340 -0.372837 32.098557 0.244696 5.588583 0.332920 -23.804396 0.000000 0.000000 0.000000 1.000000"],
		["6.168086 1.087582 -0.140082 10.105734 -0.000000 6.660676 0.229835 -10.088434 2.050541 -3.271477 0.421370 -29.995932 0.000000 0.000000 0.000000 1.000000"],
		["6.168086 -0.108740 -0.157567 11.329707 -0.000000 7.492075 -0.022980 7.608574 2.050541 0.327092 0.473967 -33.677677 0.000000 0.000000 0.000000 1.000000"],
		["5.463686 0.033567 0.270844 -18.659111 0.000000 7.499744 -0.004131 6.289180 -3.521098 0.052087 0.420269 -29.918848 0.000000 0.000000 0.000000 1.000000"],
		["5.463686 2.060403 0.233439 -16.040749 0.000000 6.463986 -0.253569 23.749834 -3.521098 3.197128 0.362228 -25.855938 0.000000 0.000000 0.000000 1.000000"],
		["5.470312 -0.977385 -0.262083 18.645786 0.000001 7.278426 -0.120637 14.444611 3.510796 1.522902 0.408362 -29.085323 0.000000 0.000000 0.000000 1.000000"],
		["5.470313 -2.812058 -0.194391 13.907358 -0.000000 5.398524 -0.347089 30.296238 3.510794 4.381584 0.302888 -21.702185 0.000000 0.000000 0.000000 1.000000"],
		["6.340922 -1.151383 0.078709 -5.209615 0.000000 5.369395 0.349092 -18.436405 -1.429231 -5.108221 0.349199 -24.943943 0.000000 0.000000 0.000000 1.000000"],
		["6.340923 -0.406811 0.106543 -7.158025 0.000000 7.268218 0.123342 -2.633968 -1.429231 -1.804858 0.472689 -33.588253 0.000000 0.000000 0.000000 1.000000"],
		["5.923949 1.699015 0.171810 -11.726667 0.000000 6.261692 -0.275207 25.264473 -2.675224 3.762256 0.380451 -27.131548 0.000000 0.000000 0.000000 1.000000"],
		["5.923949 2.759961 0.092158 -6.151034 0.000000 3.358736 -0.447059 37.294132 -2.675224 6.111588 0.204072 -14.785010 0.000000 0.000000 0.000000 1.000000"],
		["4.525468 1.813873 -0.337927 23.954880 0.000001 7.061496 0.168461 -5.792248 4.665848 -1.759300 0.327760 -23.443186 0.000000 0.000000 0.000000 1.000000"],
		["4.525468 -0.963590 -0.353116 25.018101 0.000000 7.378890 -0.089492 12.264431 4.665848 0.934599 0.342492 -24.474415 0.000000 0.000000 0.000000 1.000000"],
		["4.304458 -2.122202 0.346912 -23.983856 -0.000000 6.944674 0.188815 -7.217054 -4.870487 -1.875568 0.306596 -21.961679 0.000000 0.000000 0.000000 1.000000"],
		["4.304458 0.763958 0.371175 -25.682243 0.000000 7.430377 -0.067970 10.757923 -4.870487 0.675174 0.328038 -23.462687 0.000000 0.000000 0.000000 1.000000"],
		["5.629382 -2.503723 -0.186078 13.325452 0.000000 5.582982 -0.333868 29.370789 3.249624 4.337244 0.322346 -23.064222 0.000000 0.000000 0.000000 1.000000"],
		["5.629382 -3.563872 -0.077691 5.738356 0.000001 2.330994 -0.475238 39.266651 3.249624 6.173758 0.134585 -9.920959 0.000000 0.000000 0.000000 1.000000"],
		["5.920268 2.619186 -0.110075 8.005277 -0.000000 3.999592 0.422970 -23.607883 2.683361 -5.778679 0.242858 -17.500065 0.000000 0.000000 0.000000 1.000000"],
		["5.920268 1.442716 -0.182634 13.084397 0.000000 6.636021 0.232983 -10.308805 2.683361 -3.183048 0.402944 -28.706070 0.000000 0.000000 0.000000 1.000000"],
		["3.675664 2.170933 0.386147 -26.730310 0.000000 7.022925 -0.175481 18.283640 -5.360923 1.488479 0.264758 -19.033072 0.000000 0.000000 0.000000 1.000000"],
		["3.675665 4.776186 0.262049 -18.043432 -0.000000 4.765929 -0.386068 33.024765 -5.360922 3.274746 0.179671 -13.076999 0.000000 0.000000 0.000000 1.000000"],
		["3.107381 -0.680481 -0.436815 30.877024 0.000000 7.459877 -0.051650 9.615475 5.709131 0.370373 0.237751 -17.142540 0.000000 0.000000 0.000000 1.000000"],
		["3.107382 -3.865420 -0.355610 25.192696 0.000000 6.073073 -0.293392 26.537466 5.709131 2.103881 0.193552 -14.048662 0.000000 0.000000 0.000000 1.000000"],
		["4.135761 -4.599971 0.233980 -16.078588 -0.000000 4.549389 0.397509 -21.825649 -5.014527 -3.793853 0.192976 -14.008342 0.000000 0.000000 0.000000 1.000000"],
		["4.135762 -2.228845 0.355965 -24.617537 0.000001 6.921205 0.192607 -7.482487 -5.014526 -1.838253 0.293584 -21.050892 0.000000 0.000000 0.000000 1.000000"],
		["6.216394 1.917989 0.070641 -4.644885 -0.000000 3.626800 -0.437652 36.635635 -1.899063 6.278347 0.231237 -16.686594 0.000000 0.000000 0.000000 1.000000"],
		["-6.216394 -2.190839 -0.002756 0.492908 0.000003 0.141480 -0.499911 40.993771 1.899065 -7.171486 -0.009020 0.131465 0.000000 0.000000 0.000000 1.000000"],
		["2.830672 4.533963 -0.333503 23.645176 -0.000000 5.557174 0.335777 -17.504354 5.851264 -2.193399 0.161339 -11.793720 0.000000 0.000000 0.000000 1.000000"],
		["2.830671 1.425259 -0.439954 31.096760 -0.000000 7.330977 0.105552 -1.388638 5.851265 -0.689498 0.212837 -15.398581 0.000000 0.000000 0.000000 1.000000"],
		["2.352269 -4.290216 -0.368042 26.062958 0.000000 5.922018 -0.306809 27.476658 6.059442 1.665457 0.142874 -10.501157 0.000000 0.000000 0.000000 1.000000"],
		["2.352269 -6.475752 -0.175727 12.600883 -0.000001 2.827549 -0.463105 38.417370 6.059442 2.513881 0.068217 -5.275190 0.000000 0.000000 0.000000 1.000000"],
		["6.373148 -1.459272 0.014081 -0.685663 0.000001 1.074344 0.494844 -28.639053 -1.277881 -7.277796 0.070225 -5.415778 0.000000 0.000000 0.000000 1.000000"],
		["6.373148 -1.158160 0.060837 -3.958575 0.000000 4.641737 0.392736 -21.491495 -1.277881 -5.776067 0.303410 -21.738705 0.000000 0.000000 0.000000 1.000000"],
		["1.819810 5.001680 0.345280 -23.869604 0.000000 5.394952 -0.347336 30.313511 -6.240056 1.458658 0.100695 -7.548669 0.000000 0.000000 0.000000 1.000000"],
		["1.819811 6.921181 0.132299 -8.960909 0.000001 2.067149 -0.480633 39.644341 -6.240055 2.018451 0.038583 -3.200793 0.000000 0.000000 0.000000 1.000000"],
		["0.805984 -4.745923 0.382165 -26.451584 0.000001 5.777067 0.318856 -16.319895 -6.449836 -0.593059 0.047756 -3.842936 0.000000 0.000000 0.000000 1.000000"],
		["0.805986 -1.243851 0.489162 -33.941372 0.000000 7.394503 0.083568 0.150218 -6.449836 -0.155434 0.061127 -4.778876 0.000000 0.000000 0.000000 1.000000"],
		["1.749547 -6.863903 -0.149987 10.799120 0.000001 2.336022 -0.475128 39.258968 6.260118 1.918289 0.041918 -3.434242 0.000000 0.000000 0.000000 1.000000"],
		["-1.749547 7.069221 0.098904 -6.623261 0.000003 1.540405 -0.489340 40.253830 -6.260118 -1.975670 -0.027641 1.434879 0.000000 0.000000 0.000000 1.000000"],
		["0.266641 6.800706 -0.209822 14.987573 -0.000000 3.149989 0.453762 -25.763372 6.494529 -0.279212 0.008615 -1.103018 0.000000 0.000000 0.000000 1.000000"],
		["0.266641 4.315919 -0.408402 28.888123 -0.000000 6.131186 0.287970 -14.157923 6.494529 -0.177196 0.016767 -1.673723 0.000000 0.000000 0.000000 1.000000"],
		["-1.669135 -6.877375 0.152644 -10.385056 -0.000001 2.369097 0.474400 -27.207977 -6.282037 1.827316 -0.040557 2.339015 0.000000 0.000000 0.000000 1.000000"],
		["-1.669134 -4.811156 0.361439 -25.000736 0.000000 5.609695 0.331872 -17.231064 -6.282037 1.278322 -0.096034 6.222393 0.000000 0.000000 0.000000 1.000000"],
		["-2.549825 6.181698 0.204185 -13.992978 -0.000001 3.329668 -0.448024 37.361702 -5.978996 -2.636271 -0.087078 5.595436 0.000000 0.000000 0.000000 1.000000"],
		["2.549825 -6.884898 -0.029227 2.345869 -0.000002 0.476602 -0.498989 40.929260 5.978996 2.936160 0.012464 -1.372489 0.000000 0.000000 0.000000 1.000000"],
		["-2.947507 -5.059590 -0.291235 20.686420 -0.000000 4.901424 -0.378453 32.491730 5.793289 -2.574216 -0.148174 9.872185 0.000000 0.000000 0.000000 1.000000"],
		["-2.947505 -6.565994 -0.083564 6.149453 -0.000003 1.406361 -0.491131 40.379166 5.793291 -3.340639 -0.042516 2.476077 0.000000 0.000000 0.000000 1.000000"],
		["-5.750979 3.372309 -0.061261 4.588279 -0.000001 1.971776 0.482411 -27.768780 3.029230 6.402314 -0.116304 7.641276 0.000000 0.000000 0.000000 1.000000"],
		["-5.750979 2.461048 -0.165464 11.882480 0.000000 5.325689 0.352055 -18.643820 3.029230 4.672288 -0.314133 21.489283 0.000000 0.000000 0.000000 1.000000"],
		["-2.754611 3.401601 0.392014 -27.140997 0.000001 6.492005 -0.250367 23.525726 -5.887454 -1.591534 -0.183415 12.339033 0.000000 0.000000 0.000000 1.000000"],
		["-2.754611 5.885979 0.226108 -15.527535 0.000000 3.744486 -0.433225 36.325729 -5.887454 -2.753921 -0.105791 6.905353 0.000000 0.000000 0.000000 1.000000"],
		["-3.722164 -4.463465 0.281915 -19.434061 0.000000 5.158201 0.362969 -19.407831 -5.328743 3.117762 -0.196920 13.284378 0.000000 0.000000 0.000000 1.000000"],
		["-3.722164 -1.751111 0.392928 -27.204948 0.000002 7.189400 0.142400 -3.968035 -5.328743 1.223165 -0.274463 18.712399 0.000000 0.000000 0.000000 1.000000"],
		["-6.499726 -0.057722 -0.002499 0.474905 0.000001 4.084336 -0.419355 35.354855 0.059646 -6.290061 -0.272278 18.559431 0.000000 0.000000 0.000000 1.000000"],
		["-6.499726 -0.068728 -0.000240 0.316786 0.000001 0.391987 -0.499317 40.952168 0.059646 -7.489434 -0.026131 1.329153 0.000000 0.000000 0.000000 1.000000"],
		["-4.302106 3.945449 -0.267020 18.991407 -0.000001 5.343071 0.350882 -18.561745 4.872564 3.483536 -0.235759 16.003099 0.000000 0.000000 0.000000 1.000000"],
		["-4.302106 1.414211 -0.362761 25.693283 -0.000000 7.258850 0.125770 -2.803936 4.872564 1.248641 -0.320291 21.920351 0.000000 0.000000 0.000000 1.000000"],
		["-3.592302 -0.063625 0.416681 -28.867689 0.000001 7.499611 0.005090 5.643722 -5.417137 0.042194 -0.276317 18.842157 0.000000 0.000000 0.000000 1.000000"],
		["-3.592302 3.070007 0.362977 -25.108427 0.000001 6.533027 -0.245579 23.190540 -5.417137 -2.035832 -0.240704 16.349255 0.000000 0.000000 0.000000 1.000000"],
		["-4.405420 -2.257416 -0.335429 23.780056 -0.000000 6.842833 -0.204675 20.327229 4.779359 -2.080795 -0.309185 21.142960 0.000000 0.000000 0.000000 1.000000"],
		["-4.405420 -4.470700 -0.215243 15.367023 -0.000001 4.391007 -0.405348 34.374352 4.779359 -4.120910 -0.198402 13.388168 0.000000 0.000000 0.000000 1.000000"],
		["-6.349392 -1.308956 0.061936 -4.035504 -0.000001 4.340916 0.407739 -22.541719 -1.391122 5.974369 -0.282689 19.288223 0.000000 0.000000 0.000000 1.000000"],
		["-6.349392 -0.669070 0.097270 -6.508885 0.000001 6.817382 0.208415 -8.589063 -1.391120 3.053792 -0.443961 30.577297 0.000000 0.000000 0.000000 1.000000"],
		["-5.611467 2.101793 0.209866 -14.390611 0.000001 6.237510 -0.277637 25.434563 -3.280463 -3.595266 -0.358991 24.629345 0.000000 0.000000 0.000000 1.000000"],
		["-5.611467 3.394199 0.111689 -7.518267 0.000001 3.319571 -0.448357 37.385010 -3.280463 -5.806019 -0.191053 12.873706 0.000000 0.000000 0.000000 1.000000"],
		["-4.724389 1.390435 -0.330662 23.446341 -0.000000 7.221604 0.134964 -3.447470 4.464320 1.471434 -0.349925 23.994741 0.000000 0.000000 0.000000 1.000000"],
		["-4.724390 -1.275813 -0.332709 23.589668 -0.000000 7.266322 -0.123838 14.668653 4.464319 -1.350136 -0.352092 24.146418 0.000000 0.000000 0.000000 1.000000"],
		["-5.144022 -1.748446 0.282558 -19.479095 0.000001 6.933223 0.190676 -7.347342 -3.973541 2.263484 -0.365791 25.105392 0.000000 0.000000 0.000000 1.000000"],
		["-5.144022 0.604988 0.302984 -20.908909 -0.000001 7.434419 -0.065977 10.618374 -3.973541 -0.783200 -0.392234 26.956383 0.000000 0.000000 0.000000 1.000000"],
		["-6.284044 -0.946586 -0.111147 8.080308 -0.000001 6.522088 -0.246868 23.280769 1.661562 -3.579993 -0.420360 28.925194 0.000000 0.000000 0.000000 1.000000"],
		["-6.284044 -1.653371 -0.064704 4.829248 -0.000001 3.796786 -0.431197 36.183784 1.661562 -6.253062 -0.244709 16.629658 0.000000 0.000000 0.000000 1.000000"],
		["-6.310709 1.042817 -0.097548 7.128356 -0.000000 6.107615 0.290188 -14.313171 1.557224 4.226060 -0.395317 27.172167 0.000000 0.000000 0.000000 1.000000"],
		["-6.310709 0.171497 -0.119240 8.646772 -0.000001 7.465759 0.047723 2.659365 1.557224 0.695003 -0.483223 33.325607 0.000000 0.000000 0.000000 1.000000"],
		["-5.828282 0.396004 0.219781 -15.084682 -0.000000 7.446470 -0.059632 10.174221 -2.877695 -0.802040 -0.445130 30.659069 0.000000 0.000000 0.000000 1.000000"],
		["-5.828282 1.991309 0.177136 -12.099519 0.000001 6.001594 -0.299858 26.990070 -2.877694 -4.033058 -0.358759 24.613125 0.000000 0.000000 0.000000 1.000000"],
		["-6.010379 -0.084492 -0.190297 13.620784 -0.000001 7.496716 -0.014793 7.035545 2.474942 -0.205187 -0.462134 31.849415 0.000000 0.000000 0.000000 1.000000"],
		["-6.010379 -1.500397 -0.161986 11.638994 0.000001 6.381398 -0.262702 24.389139 2.474944 -3.643702 -0.393381 27.036648 0.000000 0.000000 0.000000 1.000000"],
		["-6.428964 -0.362217 0.069651 -4.575593 -0.000001 7.086210 0.163783 -5.464835 -0.958341 2.429902 -0.467251 32.207588 0.000000 0.000000 0.000000 1.000000"],
		["-6.428964 0.208695 0.072394 -4.767558 -0.000001 7.365213 -0.094366 12.605642 -0.958341 -1.400027 -0.485648 33.495373 0.000000 0.000000 0.000000 1.000000"],
		["-6.500000 0.000001 0.000000 0.300000 0.000001 7.471460 0.043578 2.949553 -0.000001 0.653667 -0.498097 34.366814 0.000000 0.000000 0.000000 1.000000"],
		["-6.500000 0.000001 -0.000000 0.300000 0.000001 6.797308 -0.211309 20.791634 -0.000000 -3.169636 -0.453154 31.220776 0.000000 0.000000 0.000000 1.000000"],
	]
	raw_string = light_positions[camera_idx][0]
	float_values = list(map(float, raw_string.strip().split()))
	matrix_4x4 = np.array(float_values).reshape((4, 4))
	params['arealight.to_world'] = mi.Transform4f(matrix_4x4)
	params.update()

def save_images(scene, params, dir, spp):
    print("start rendering")
    for i, sensor in enumerate(scene.sensors()):
					_update_light_position(params, i)
											
					integrator = mi.load_dict({
							'type': 'prb_nee_volume',
							'max_sss_depth': -1,
							'max_path_depth': 256,
					})
					image = mi.render(scene, sensor=sensor, integrator=integrator, spp=spp)
					bitmap = mi.Bitmap(image).convert(
									pixel_format=mi.Bitmap.PixelFormat.RGB,
									component_format=mi.Struct.Type.UInt8,
									srgb_gamma=True
					)
					png_path = dir / f"ref_view_{i:03d}.png"
					bitmap.write(str(png_path))

					bitmap = mi.Bitmap(image).convert(
													pixel_format=mi.Bitmap.PixelFormat.RGB,
													component_format=mi.Struct.Type.Float32,
													srgb_gamma=False
					)
					exr_path = dir / f"ref_view_{i:03d}.exr"
					bitmap.write(str(exr_path))
					print(f"saving: {i}")

In [ ]:
# Display reference image
scene_name = 'kiwi_refine'
technique = 'mipmap_pyramid'
result_folder = f'results/{scene_name}/{technique}'
scene_folder = f'third_party/{scene_name}'
result_folder += '_test'

override_bindings = []
override_bindings.append(f"SceneConfig.result_folder='{result_folder}'")
override_bindings.append(f"SceneConfig.scene_folder='{scene_folder}'")
override_bindings.append(f'SceneConfig.base_learning_rate={0.1}')
scene_config = scene_configuration.SceneConfig.get_instance(
				f'{scene_name}/{technique}', override_bindings, sss_config=True
)

scene = load_modified_scene(scene_config,res=[800,480])
ref_spp=4096
opt_spp=128
params = mi.traverse(scene)

ref_dir=Path("third_party/kiwi_refine/references")
ref_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_refine_ref.xml'
ref_scene = mi.load_file(ref_scene_path)
ref_params = mi.traverse(ref_scene)
save_images(ref_scene, ref_params, ref_dir, ref_spp)

test_dir=Path("third_party/kiwi_refine/test")
test_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_test.xml'
test_scene = mi.load_file(test_scene_path)
test_params = mi.traverse(test_scene)
#print(test_scene)
#save_images(test_scene, test_params, test_dir, ref_spp)

start rendering
saving: 0
saving: 1
saving: 2
saving: 3
saving: 4
saving: 5
saving: 6
saving: 7
saving: 8
saving: 9
saving: 10
saving: 11
saving: 12
saving: 13
saving: 14
saving: 15
saving: 16
saving: 17
saving: 18
saving: 19
saving: 20
saving: 21
saving: 22


In [ ]:
if technique == 'gradient_filtering':
    gin_config_name = f'{scene_name}/naive'
else:
    gin_config_name = f'{scene_name}/{technique}'

print(f'Next result location: {result_folder}')

print(scene_config)
optimization_cli.run_config(gin_config_name, override_bindings, sss_config=True)

In [ ]:
# Figure setup

def figure_grid_setup(
    fig_width,
    image_shape,
    image_crop_shape,
    inner_hspace=0.0,
    inner_wspace=0.0,
    outer_space=0.1,
):
  # Image aspect ratios
  h, w = image_shape
  h_crop, w_crop = image_crop_shape
  r = w / h
  r_crop = w_crop / h_crop

  # Spacing in the inner gridspec
  inner_wspace = inner_hspace
  # same vertical spacing as horizontal spacing
  inner_hspace = inner_wspace
  inner_rows = 2
  inner_cols = 3
  inner_height_ratios = [h_crop, h_crop]
  inner_width_ratios = [w_crop, w_crop, w_crop]
  inner_aspect = figutils.gridspec_aspect(
      n_rows=inner_rows,
      n_cols=inner_cols,
      w=inner_width_ratios,
      h=inner_height_ratios,
      wspace=inner_wspace,
      hspace=inner_hspace,
  )
  # Spacing in the main griddpec
  outer_rows = 2
  outer_cols = 1
  outer_wspace = 0
  outer_hspace = outer_space
  # If width is 1, we need the sum of the inverses for the height (single column)
  # If height is 1, we need the sum for the width (single row)
  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=[1],
      h=[1 / inner_aspect, 1 / r],
      wspace=outer_wspace,
      hspace=outer_hspace,
  )
  outer_aspect *= 0.98

  fig = plt.figure(1, figsize=(fig_width, fig_width / outer_aspect))

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1 / inner_aspect, 1 / r],
      width_ratios=[1],
  )

  inner_gs = gridspec.GridSpecFromSubplotSpec(
      inner_rows,
      inner_cols,
      subplot_spec=outer_gs[0],
      wspace=inner_wspace,
      hspace=inner_hspace,
      width_ratios=inner_width_ratios,
      height_ratios=inner_height_ratios,
  )
  return (
      fig,
      outer_gs,
      inner_gs,
      inner_rows,
      inner_cols,
  )

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "volume_gradients"

ref_crop_offset = (26, 22)
ref_crop_size = (739, 432)

grad_crop_offset = (18, 13)
grad_crop_size = (363, 215)

cropped_ref_image = figutils.crop_image(ref_image,ref_crop_offset,ref_crop_size)
cropped_forward_images = [figutils.crop_image(im,grad_crop_offset,grad_crop_size) for im in forward_images]
cropped_fd_images = [figutils.crop_image(im,grad_crop_offset,grad_crop_size) for im in fd_images]

titles = [
    "Albedo",
    "Extinction",
    "Phase function (HG)",
    r"\textsc{Dragon}",
]
row_titles = ["Our forward", "Finite diff."]

params_label = [
    r"$\rho=[0.2, 0.4, 0.95]$",
    r"$\sigma_t=[15,7,5]$",
    r"$g=[-0.5, -0.2, 0.1]$",
]

(
    fig,
    outer_gs,
    inner_gs,
    inner_rows,
    inner_cols,
) = figure_grid_setup(
    figutils.COLUMN_WIDTH,
    ref_image.shape[:2],
    cropped_forward_images[0].shape[:2],
    inner_hspace=0.02,
    inner_wspace=0.02,
    outer_space=0.0,
)

title_pad = 3
label_pad = 2
line_width = 0.75

strengths = [4.5, 4.5, 4.5]
# Grads
if True:
  for row in range(inner_rows):
    for col in range(inner_cols):
      ax = fig.add_subplot(inner_gs[row, col])
      figutils.disable_ticks(ax)
      if col == 0:
        ax.set_ylabel(row_titles[row], labelpad=label_pad)
      # Our
      if row == 0:
        ax.set_title(titles[col], pad=title_pad)
        im = ax.imshow(
            cropped_forward_images[col],
            cmap=cm.coolwarm,
            vmin=-forward_vlims[col] * strengths[col],
            vmax=forward_vlims[col] * strengths[col],
        )
      elif row == 1:
        im = ax.imshow(
            cropped_fd_images[col],
            cmap=cm.coolwarm,
            vmin=-forward_vlims[col] * strengths[col],
            vmax=forward_vlims[col] * strengths[col],
        )

# Reference
if True:
  ax = fig.add_subplot(outer_gs[1])
  im = ax.imshow(cropped_ref_image)
  ax.set_ylabel(titles[-1], labelpad=label_pad)
  # ax.set_title(r"\vspace{250px}" + r"\;\;\;\;\;\;\;\;\;".join(params_label),pad=0)
  ax.set_xlabel(r"\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;".join(params_label),labelpad=label_pad)
  figutils.disable_ticks(ax)

  ax = fig.add_subplot(outer_gs[0])
  ax.axis('off')
  # Create an inset axes
  inset = inset_axes(
      ax,  # Parent axes
      width="1%",  # Width of the inset in percentage of the parent axis
      height="99.19%",  # Height of the inset in percentage of the parent axis
      loc="lower left",  # Location inside the parent axis
      bbox_to_anchor=(1.005, 0.005, 1, 1),  # Bounding box adjustment
      bbox_transform=ax.transAxes,  # Use axes coordinates
      borderpad=0,
  )  # Padding

  # Create the colorbar
  cbar = fig.colorbar(
      cm.ScalarMappable(
          norm=Normalize(
              vmin=-np.max(forward_vlims), vmax=np.max(forward_vlims)
          ),
          cmap=cm.coolwarm,
      ),
      cax=inset,
      orientation="vertical",
  )
  cbar.ax.yaxis.set_label_position("left")  # Move label to the left
  # cbar.ax.yaxis.tick_left()  # Move ticks to the left side
  cbar.set_label("Parameter gradient", labelpad=-9)
  ticks = [cbar.vmin, cbar.vmax]
  cbar.set_ticks(ticks)
  # cbar.set_ticklabels([f"{x:.2f}" for x in ticks])
  cbar.set_ticklabels(["neg", "pos"])
  cbar.ax.tick_params(pad=1.8, length=0)

# fig.tight_layout()
figutils.force_post_crop_size(fig, figutils.COLUMN_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.COLUMN_WIDTH,
    backend=None,
)